<a href="https://colab.research.google.com/github/beingnator/ECMI-hydrological-forecasting/blob/main/HydroProjectECMI_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pytorch CNN

In [ ]:
#Importing libraries
import seaborn as sns
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

#Manually setting a seed for random generators. This way the results can be reproduced.
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
#Downloading the created training and validation files and maximum correlation values for the spatial model
!gdown --id 1t2xlzVbw_Oc6pYrUk5ntQP55shUllgRK
!gdown --id 1xjTRjbiBVltExeSKePUyFrP-UvpJpbAb
!gdown --id 14tlqDh2cJG75sQO-uTmB9NcGOxKB7c5l

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1t2xlzVbw_Oc6pYrUk5ntQP55shUllgRK
To: /content/data_training.csv
100% 7.13M/7.13M [00:00<00:00, 90.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1xjTRjbiBVltExeSKePUyFrP-UvpJpbAb
To: /content/data_validation.csv
100% 1.71M/1.71M [00:00<00:00, 134MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.

In [ ]:
#Reading the files containing the preprocessed training and validation datas, including a split.
#The split makes sure that there is at least one flood in both dataset.
data_train = pd.read_csv("data_training.csv",index_col=0)
data_valid = pd.read_csv("data_validation.csv",index_col=0)
max_corr = pd.read_csv("maximum_correlation.csv",index_col=0)
#Changing the data format of the index columns.
data_train.index = pd.to_datetime(data_train.index)
data_valid.index = pd.to_datetime(data_valid.index)
#Sorting the columns according to the distance between the stations
#calculated from the maximal correlations.
max_corr.columns = ["Distance"]
data_train = data_train[list(map(str, max_corr.sort_values(by = ["Distance"]).index.to_list()))]
data_valid = data_valid[list(map(str, max_corr.sort_values(by = ["Distance"]).index.to_list()))]

In [ ]:
colnum = len(data_train.columns.tolist()) #Number of stations
pastWindow = 15 #Number of days before prediction
windowSize = 7 #Number of predicted days

In [ ]:
#This function requires the dataset, the number of days before the prediction and the number of the columns
#and gives back a torch tensor with a sliding window along time, selecting pastWindow size tensors.
def gen_X_tensor(dataset, pastWindow, colnum):
  windowed_data = np.lib.stride_tricks.sliding_window_view(dataset, (pastWindow,colnum)).reshape((-1,pastWindow,colnum))
  return torch.tensor(windowed_data, dtype = torch.float32)

#This function requires the dataset and the number of days, which needed to be predicted.
#Gives back the following windowSize days as the ground truth water level in Szeged.
def gen_y_tensor(dataset, windowSize):
  y_vals = []
  for i in range(len(dataset)-windowSize+1):
    days = []
    for j in range(windowSize):
      days.append(dataset["2275"].values[i+j])
    y_vals.append(days)
  return torch.reshape(torch.tensor(y_vals, dtype = torch.float32),(-1,windowSize))

In [ ]:
#Creating the training/validation input datas. Because the predicted ground truth days
#should be from the training/validation datas as well, the last windowSize days cannot be
#included into the training/validation data.
X_train = gen_X_tensor(data_train[:-windowSize], pastWindow, colnum)
X_valid = gen_X_tensor(data_valid[:-windowSize], pastWindow, colnum)

CNNnumpy = torch.Tensor(data_train.to_numpy()) #Tensor from pandas dataframe
means = CNNnumpy.mean(axis=0, keepdims=True) #Mean of the training data
stds = CNNnumpy.std(axis=0, keepdims=True) #Standard deviation of the training data
X_train = (X_train - means) / stds #Standardized training input
X_valid = (X_valid - means) / stds #Standardized validation input

#Generation of the ground truth output vectors, with windowSize. The first pastWindow size
#cannot be used, because there is not enough past size before that.
y_train = gen_y_tensor(data_train[pastWindow:],windowSize)
y_valid = gen_y_tensor(data_valid[pastWindow:],windowSize)
#Mean and Standard Deviation of Szeged.
y_mean = torch.full((1,7),means[0,3])
y_stds = torch.full((1,7),stds[0,3])
#Standardization of the training and validation datas.
y_train = (y_train - y_mean) / y_stds
y_valid = (y_valid - y_mean) / y_stds
#Check if the number of the input and output equal to each other.
print(f"Number of input: {len(X_train)}\t Number of output: {len(y_train)}")
#Change dimensions to be able to use them in the CNN network.
X_train = torch.transpose(X_train,1,2)
X_valid = torch.transpose(X_valid,1,2)

Number of input: 19705	 Number of output: 19705


In [ ]:
#Predefinition of the increasing size of channels. This way we provide more and more informations
#in the layers being deeper and deeper, so the spatial information is included in a way
#that the deeper the layer is the larger region of interest it has.
channelSize = []
for i in range(7):
  channelSize.append(max_corr[max_corr.Distance <= i].count().values[0])

In [ ]:
import torch.nn as nn

#Simple CNN module with 3 1D convolutional layer. The convolution only goes along the
#time using all the stations as channels. The kernel_size is picked in a way that
#the output dimension should be allright.
class HydroForecast(nn.Module):
    #definition of the CNN layer
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(colnum, 7, kernel_size=5),
            nn.ReLU(),
            nn.Conv1d(7, 7, kernel_size=9),
            nn.ReLU(),
            nn.Conv1d(7, 7, kernel_size=3))

    #creating the forward flow
    def forward(self, x):
      x = self.model(x)
      return x.reshape(-1,7)

#Creating layers for different distances away from Szeged. The deeper the network the further the network sees.
class HydroForecastSpatialVersion(nn.Module):
    def __init__(self):
        super().__init__()
        self.Day1Model = nn.Sequential(
            nn.Conv1d(channelSize[0], channelSize[0], kernel_size=1),
            nn.ReLU())
        self.Day2Model = nn.Sequential(
            nn.Conv1d(channelSize[1], channelSize[1], kernel_size=1),
            nn.ReLU())
        self.Day3Model = nn.Sequential(
            nn.Conv1d(channelSize[2], channelSize[2], kernel_size=1),
            nn.ReLU())
        self.Day4Model = nn.Sequential(
            nn.Conv1d(channelSize[3], channelSize[3], kernel_size=1),
            nn.ReLU())
        self.Day5Model = nn.Sequential(
            nn.Conv1d(channelSize[4], channelSize[4], kernel_size=1),
            nn.ReLU())
        self.Day6Model = nn.Sequential(
            nn.Conv1d(channelSize[5], channelSize[5], kernel_size=1),
            nn.ReLU())
        self.Day7Model = nn.Sequential(
            nn.Conv1d(channelSize[6], colnum, kernel_size=1),
            nn.ReLU())
        #At the end the simplest CNN network is attached.
        self.model = nn.Sequential(
            nn.Conv1d(colnum, 7, kernel_size=5),
            nn.ReLU(),
            nn.Conv1d(7, 7, kernel_size=9),
            nn.ReLU(),
            nn.Conv1d(7, 7, kernel_size=3))

    #Building the forward flow of the network.
    def forward(self, x):
      xw = x[:,0:channelSize[0],:]
      xw = self.Day1Model(xw)
      xw = torch.cat((xw, x[:,channelSize[0]:channelSize[1]]),1)
      xw = self.Day2Model(xw)
      xw = torch.cat((xw, x[:,channelSize[1]:channelSize[2]]),1)
      xw = self.Day3Model(xw)
      xw = torch.cat((xw, x[:,channelSize[2]:channelSize[3]]),1)
      xw = self.Day4Model(xw)
      xw = torch.cat((xw, x[:,channelSize[3]:channelSize[4]]),1)
      xw = self.Day5Model(xw)
      xw = torch.cat((xw, x[:,channelSize[4]:channelSize[5]]),1)
      xw = self.Day6Model(xw)
      xw = torch.cat((xw, x[:,channelSize[5]:channelSize[6]]),1)
      xw = self.Day7Model(xw)
      xw = self.model(xw)
      return xw.reshape(-1,7)

In [ ]:
#Definition of loss function, by adding weights to stations in a way that:
#The closer the station is in time, the bigger the loss is.
def mylossW(output, target, weight):
  loss = torch.mean((torch.div(output - target,weight[:output.shape[0]]))**2)
  return loss

#Linear definition of the weigths. The weights according to these settings are linearly increasing numbers from 1 to the end of the predicted timeperiod.
w = []
for batch_iter in range(0,len(y_train)):
    wdays = []
    w.append(np.linspace(1,1,windowSize))
w =torch.Tensor(w)

<ipython-input-13-f118007eb277>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  w =torch.Tensor(w)


In [ ]:
def my_loss1(output, target, weight): #custom relu
  relu_breakpoint_Y = 0.4   #what weight all the small water-level datas get
  relu_breakpoint_X = 0.3   #where the weight-increasing starts

  loss = torch.mean((torch.div((output - target)*(        (target < relu_breakpoint_X) * relu_breakpoint_Y + (target >= relu_breakpoint_X) * (relu_breakpoint_Y + target - relu_breakingpoint_X)        ),weight[:output.shape[0]]))**2)
  return loss

In [ ]:
def my_loss2(output, target, weight): #custom relu squared
  relu_breakpoint_Y = 0.4   #what weight all the small water-level datas get
  relu_breakpoint_X = 0.3   #where the weight-increasing starts

  loss = torch.mean((torch.div((output - target)*(        (target < relu_breakpoint_X) * relu_breakpoint_Y + (target >= relu_breakpoint_X) * (relu_breakpoint_Y + target - relu_breakingpoint_X)**2        ),weight[:output.shape[0]]))**2)
  return loss

In [ ]:
def my_loss3(output, target, weight): #custom two section squared If[x < 0, -x^2/5 + 1, x^2 + 1]

  loss = torch.mean((torch.div((output - target)*(        (target < 0) *((target**2) * (-0.2) + 1) + (target >= 0) * (target**2 + 1)     ),weight[:output.shape[0]]))**2)
  return loss

In [ ]:
def my_loss4(output, target, weight): #custom staircase function
  starting_value = 0.5

  step1x = 0.2
  step1y = 0.7

  step2x = 0.5
  step2y = 1

  step3x = 0.8
  step3y = 1.5

  loss = torch.mean((torch.div((output - target)*(      (target < step1x)*starting_value + (step1x < target)*(target < step2x)*step1y + (step2x < target)*(target < step3x)*step2y + (step3x < target)*step3y          ),weight[:output.shape[0]]))**2)
  return loss

In [ ]:
#Transformation back from the standardized form of data.
def eval_back(y_vals, means, sdts):
  return y_vals*sdts+means

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data

#Manually setting a seed for random generators. This way the results can be reproduced.
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Make an instance from the deep learning model.
model = HydroForecast()
#Alias for myloss to specify which loss function should be used
my_loss = mylossW
#Define batchSize
batchSize = 8

#Adam optimizer creation.
optimizer = optim.Adam(model.parameters())
#Scheduler to reduce learnin rate on plateau.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

#Data loader
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batchSize)

n_epochs = 500 #Number of epochs
last_loss = 100 #Initial last_loss value for early stopping.
patience = 10 #Patience of early stopping
trigger_times = 0 #How many times was worse or result than before?

#Training
for epoch in range(n_epochs):
    model.train() #switch to train mode
    #For every batch, making prediction, calculating loss and then use optimizer.
    #This is the standard workflow of Pytorch.
    for X_batch, y_batch in loader: #Loading data in batches
        y_pred = model(X_batch) #make prediction
        loss = my_loss(y_pred, y_batch,w) #calculate loss
        optimizer.zero_grad() #zero the optimizer
        loss.backward() #calculate backpropogation
        optimizer.step() #make an optimization step
    model.eval() #switch evaluation mode
    with torch.inference_mode(): #inference_mode on
        #Loss of training and loss of validation is calculated here.
        #Standard workflow for validation
        y_pred = model(X_train) #make prediction in training
        train_rmse = np.sqrt(my_loss(y_pred, y_train,w)) #calculate loss
        y_pred = model(X_valid) #make prediction in validation
        valid_rmse = np.sqrt(my_loss(y_pred, y_valid,w)) #calculate loss
        print(f"Prediction: {eval_back(y_pred[1],y_mean,y_stds)}\nGround truth: {eval_back(y_valid[1],y_mean,y_stds)}")
    print(f"Epoch %d: train loss %.4f, validation loss %.4f" % (epoch, train_rmse, valid_rmse))
     # Early stopping
    current_loss = valid_rmse #Value of the current loss
    print('Current Loss: %.4f\t Last Loss: %.4f' % (current_loss,last_loss))

    #If current loss is worse or close to the current loss, trigger
    if last_loss-current_loss < 1e-5:
      trigger_times += 1
      print(f'Trigger Times: {trigger_times}')
    else:
      trigger_times = 0 #reset trigger

    #If the number of trigger times is larger of equal to patience number training over
    if trigger_times >= patience:
      print('Early stopping!\nStart to test process.')
      break
    print(f'trigger times: {trigger_times}')

    #Last loss equals to current loss
    last_loss = current_loss
    #Giving validation loss for the scheduler
    scheduler.step(valid_rmse)

Prediction: tensor([[219.0875, 207.2457, 202.5524, 195.3435, 200.3463, 194.8847, 202.2698]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 0: train loss 0.2208, validation loss 0.1644
Current Loss: 0.1644	 Last Loss: 100.0000
trigger times: 0
Prediction: tensor([[227.7005, 217.0566, 207.0986, 198.3255, 202.6322, 198.8045, 197.6911]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 1: train loss 0.1906, validation loss 0.1610
Current Loss: 0.1610	 Last Loss: 0.1644
trigger times: 0
Prediction: tensor([[230.6113, 222.8129, 202.9823, 192.9484, 194.6971, 190.0906, 191.7799]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 2: train loss 0.1880, validation loss 0.1789
Current Loss: 0.1789	 Last Loss: 0.1610
Trigger Times: 1
trigger times: 1
Prediction: tensor([[224.3142, 211.2612, 188.8936, 176.8887, 173.2155, 166.0310, 170.1372]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 3: train lo

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data

#Manually setting a seed for random generators. This way the results can be reproduced.
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Make an instance from the deep learning model.
modelSpat = HydroForecastSpatialVersion()
#Alias for myloss to specify which loss function should be used
my_loss = mylossW
#Define batchSize
batchSize = 8

#Adam optimizer creation.
optimizer = optim.Adam(modelSpat.parameters())
#Scheduler to reduce learnin rate on plateau.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

#Data loader
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batchSize)

n_epochs = 500 #Number of epochs
last_loss = 100 #Initial last_loss value for early stopping.
patience = 10 #Patience of early stopping
trigger_times = 0 #How many times was worse or result than before?

#Training
for epoch in range(n_epochs):
    modelSpat.train() #switch to train mode
    #For every batch, making prediction, calculating loss and then use optimizer.
    #This is the standard workflow of Pytorch.
    for X_batch, y_batch in loader: #Loading data in batches
        y_pred = modelSpat(X_batch) #make prediction
        loss = my_loss(y_pred, y_batch,w) #calculate loss
        optimizer.zero_grad() #zero the optimizer
        loss.backward() #calculate backpropogation
        optimizer.step() #make an optimization step
    modelSpat.eval() #switch evaluation mode
    with torch.inference_mode(): #inference_mode on
        #Loss of training and loss of validation is calculated here.
        #Standard workflow for validation
        y_pred = modelSpat(X_train) #make prediction in training
        train_rmse = np.sqrt(my_loss(y_pred, y_train,w)) #calculate loss
        y_pred = modelSpat(X_valid) #make prediction in validation
        valid_rmse = np.sqrt(my_loss(y_pred, y_valid,w)) #calculate loss
        print(f"Prediction: {eval_back(y_pred[1],y_mean,y_stds)}\nGround truth: {eval_back(y_valid[1],y_mean,y_stds)}")
    print(f"Epoch %d: train loss %.4f, validation loss %.4f" % (epoch, train_rmse, valid_rmse))
     # Early stopping
    current_loss = valid_rmse #Value of the current loss
    print('Current Loss: %.4f\t Last Loss: %.4f' % (current_loss,last_loss))

    #If current loss is worse or close to the current loss, trigger
    if last_loss-current_loss < 1e-5:
      trigger_times += 1
      print(f'Trigger Times: {trigger_times}')
    else:
      trigger_times = 0 #reset trigger

    #If the number of trigger times is larger of equal to patience number training over
    if trigger_times >= patience:
      print('Early stopping!\nStart to test process.')
      break
    print(f'trigger times: {trigger_times}')

    #Last loss equals to current loss
    last_loss = current_loss
    #Giving validation loss for the scheduler
    scheduler.step(valid_rmse)

Prediction: tensor([[214.4546, 200.2396, 198.5512, 189.6375, 186.8123, 183.5941, 179.7700]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 0: train loss 0.2275, validation loss 0.1840
Current Loss: 0.1840	 Last Loss: 100.0000
trigger times: 0
Prediction: tensor([[214.7112, 202.2579, 190.0246, 178.1639, 177.0756, 175.5991, 178.1323]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 1: train loss 0.1871, validation loss 0.1550
Current Loss: 0.1550	 Last Loss: 0.1840
trigger times: 0
Prediction: tensor([[229.2213, 215.8266, 204.9330, 195.6217, 189.2757, 187.9439, 190.1462]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 2: train loss 0.1860, validation loss 0.1560
Current Loss: 0.1560	 Last Loss: 0.1550
Trigger Times: 1
trigger times: 1
Prediction: tensor([[223.4918, 212.7811, 207.6844, 202.4059, 202.4026, 207.0791, 213.3328]])
Ground truth: tensor([[215., 202., 192., 181., 170., 163., 161.]])
Epoch 3: train lo

In [ ]:
#Function for creating pandas dataframe with the predicted 7 days. This way the model can be observed out of this notebook as well.
def get_predictions(model):
    pred = pd.DataFrame(
        data=model(X_valid).detach().numpy(),
        index=data_valid[pastWindow-1:-windowSize].index,
        columns=["1day","2day","3day","4day","5day","6day","7day"])
    return pred

#Get the dataframe for both model
predCNN = get_predictions(model)
predCNNSpat = get_predictions(modelSpat)

In [ ]:
import os
from google.colab import drive
#Mounting Colab to Google Drive.
drive.mount('/content/drive/')

#Creating directory for the models.
!mkdir -p "/content/drive/My Drive/ColabNotebooks/ECMI/Models"

#Created filename
fName = "LSTMmodel_s_loss3.csv"

#Saving prediction result file to the cloud computer.
predCNN.to_csv(fName, ',')

#Copy the created file to Drive.
!cp -r '/content/'+fName "/content/drive/My Drive/ColabNotebooks/ECMI/Models"

#Changing file name for spatial model to be able to distuingish the result
fName = fName+"Spat"

#Saving prediction result file to the cloud computer.
predCNNSpat.to_csv(fName, ',')

#Copy the created file to Drive.
!cp -r '/content/'+fName "/content/drive/My Drive/ColabNotebooks/ECMI/Models"

Mounted at /content/drive/
cp: cannot stat '/content/+fName': No such file or directory
cp: cannot stat '/content/+fName': No such file or directory


In [ ]:
# Write standardization parameters to Google Drive
# Data Mean
with open('/content/drive/My Drive/ColabNotebooks/ECMI/Models/paramsMean.csv', 'w') as f:
  mlist = ["{}\n".format(i) for i in means[0]]
  f.writelines(mlist)
# Data Standard Deviation
with open('/content/drive/My Drive/ColabNotebooks/ECMI/Models/paramsSTDS.csv', 'w') as f:
  mlist = ["{}\n".format(i) for i in stds[0]]
  f.writelines(mlist)
# Mean only for Szeged to calculate in the case of Y values
with open('/content/drive/My Drive/ColabNotebooks/ECMI/Models/paramsYMean.csv', 'w') as f:
  mlist = ["{}\n".format(i) for i in y_mean[0]]
  f.writelines(mlist)
# Standard Deviation only for Szeged to calculate in the case of Y values
with open('/content/drive/My Drive/ColabNotebooks/ECMI/Models/paramsYSTDS.csv', 'w') as f:
  mlist = ["{}\n".format(i) for i in y_stds[0]]
  f.writelines(mlist)